# Adding Layers to the NN

Ref Paper : https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf     
Ref Notebook : https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/makemore/makemore_part2_mlp.ipynb


In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [28]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
... ---> a
..a ---> v
.av ---> a
ava ---> .
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [29]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [30]:
C = torch.randn((27,2))  # 27 chars in a 2d space

In [31]:
print(C[5])
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([ 0.0234, -0.4436])


tensor([ 0.0234, -0.4436])

In [33]:
X[13,1]

tensor(0)

In [34]:
print(C[X[13,1]])
C[0]

tensor([-0.1526,  1.2262])


tensor([-0.1526,  1.2262])

In [36]:
emb = C[X]  # creating embeddings
emb.shape

torch.Size([32, 3, 2])

In [39]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [41]:
# adjusting shape 
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([32, 6])

In [44]:
h = emb.view(-1, 6) @ W1 + b1  # using view since it is computationally efficient
h.shape

torch.Size([32, 100])

In [45]:
h

tensor([[ 0.8261,  2.2743,  1.1182,  ..., -0.9410,  1.2477, -0.3422],
        [ 2.3607,  0.5881,  0.2553,  ...,  0.6048,  3.2003,  1.0931],
        [ 2.8802, -1.0586,  1.2458,  ...,  1.1670,  1.8980, -1.9208],
        ...,
        [ 0.2490, -0.7695,  1.6617,  ...,  0.6566,  0.4313, -1.7774],
        [ 0.4947, -0.5772,  0.0332,  ..., -0.9599, -0.7008, -3.3758],
        [ 1.4935,  1.1335,  1.1170,  ..., -3.5095, -0.8179, -1.8932]])

In [46]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [48]:
logits = h @ W2 + b2
logits

tensor([[ 1.3243e+01,  1.5158e+01,  1.8106e+01,  6.5567e+00,  2.8933e+01,
         -2.9779e+01, -1.3283e+01, -5.4683e+00,  2.5416e+01, -2.9753e+01,
          3.1175e+00,  4.9598e+00,  1.2845e+01, -3.2694e+01,  3.5672e+01,
          5.2572e+01,  1.6752e+01,  1.7260e+01, -6.6283e+00,  2.1063e+01,
         -1.1042e+01,  4.0754e+00,  1.7313e+01,  1.9788e+01, -2.0945e+00,
          6.1351e+01,  3.6448e+00],
        [-6.9918e-01,  1.4074e+01,  1.1429e+01,  1.4385e+01,  1.4317e+01,
         -5.9694e+00,  9.7217e+00, -3.7448e+00, -2.3136e+01, -2.2603e+01,
          1.8688e+01, -1.9699e+01,  3.4505e+01, -5.1878e+00,  1.5174e+01,
          3.3814e+01,  3.8692e+01,  1.5040e+01,  2.3890e-01,  2.1797e+01,
         -2.5811e+01,  1.6970e+01,  1.4892e+01, -1.9440e+01, -1.6381e+00,
          3.9977e+01,  3.1631e-01],
        [-3.4259e+00,  8.7256e+00,  4.3330e+00, -4.6804e+00, -2.2613e+00,
          7.7554e+00,  1.4321e+00,  1.4629e+01, -2.8161e+01, -8.4981e+00,
          2.0092e+01, -1.2437e+01,  1.45

In [49]:
logits.shape

torch.Size([32, 27])

In [50]:
counts = logits.exp()

In [55]:
prob = counts / counts.sum(1, keepdims=True)

In [56]:
prob.shape

torch.Size([32, 27])

In [57]:
prob[torch.arange(32), Y]

tensor([2.6475e-40, 1.8929e-20, 2.2048e-12, 6.4177e-26, 5.5710e-21, 1.5387e-04,
        2.6280e-04, 1.5678e-24, 4.4123e-13, 7.2238e-13, 8.8354e-30, 3.4618e-20,
        8.6774e-21, 5.9476e-28, 4.2913e-29, 8.5967e-36, 2.7161e-40, 4.7413e-11,
        3.6086e-09, 2.5047e-21, 2.7785e-36, 1.6674e-20, 6.4836e-01, 2.7954e-23,
        1.1391e-13, 3.1832e-18, 2.3807e-11, 3.8831e-14, 7.5220e-26, 3.8030e-15,
        7.2475e-29, 1.2370e-13])

In [58]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(44.8578)

# ------------ coupling and generalizing ------------

In [60]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [63]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [64]:
sum(p.nelement() for p in parameters)

3481

In [67]:
emb = C[X]  # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)   # (32, 100)
logits = h @ W2 + b2    # (32, 27)
# counts = logits.exp()
# prob = counts / counts.sum(1, keepdim=True) 
# loss = -prob[torch.arange(32), Y].log().mean()
# loss
F.cross_entropy(logits, Y)

tensor(17.7697)